In [9]:
import re

import pandas as pd
import numpy as np

In [10]:
depth = 12
df = pd.read_json(f"../data/wikipedia_depth_{depth}.json").rename(columns={'Category': 'Path'})
df['Category'] = df['Path'].apply(lambda x: x[-1])

In [11]:
flatten = lambda l: [item for sublist in l for item in sublist]

df_unique = df.groupby('Title').agg(list).reset_index()
df_unique['Links'] = df_unique['Links'].apply(flatten).apply(np.unique)
df_unique['Text'] = df_unique['Text'].apply(lambda x: list(set(x))[0])
print("Number of unique articles:", len(df_unique))

Number of unique articles: 25593


In [12]:
subset = ['Methane', 'Extinction Rebellion', 'Fuel taxes', 'Hydraulic fracturing', 'Exxonmobil', 'Gazprom', 
          'Self-sustainability', 'Industrial ecology', 'Ecovillages', 'Eco-towns', 'Wildlife smuggling', 
          'Urban forestry', 'Biofuels', 'Sustainable gardening', 'Animal waste products', 'Oil platform disasters',
          'Coal phase-out', 'Climate change denial', 'Building energy rating', 'Active fire protection', 
          'Industrial minerals', 'Composting', 'Reforestation']
subset = [w.lower() for w in subset]

df_subset = df[df['Category'].map(lambda x: x.lower()).isin(subset)].reset_index().drop(columns=['index'])
df_subset['Text'] = df_subset['Text'].apply(lambda x: re.sub('\n', '. ', x))
df_subset.shape

(937, 5)

In [13]:
# Remove punctuation
df_subset['Text_processed'] = df_subset['Text'].map(lambda x: re.sub('[,\.!?]', '', x))
df_subset['Text_processed'] = df_subset['Text'].map(lambda x: x.lower())
df_subset['Text_processed'].head()

0    climate change denial, or global warming denia...
1    habibullo ismailovich abdussamatov (russian: х...
2    accuracy in media (aim) is an american non-pro...
3    robert brown aderholt (born july 22, 1965) is ...
4    jerry agar is a conservative talk radio person...
Name: Text_processed, dtype: object

In [14]:
df_subset.to_pickle("../data/df_subset.pkl")